In [5]:
import numpy as np
import copy

In [6]:
test_cases = [
    [
        syst := [
            [-2, -1, -4, 1, 0],
            [-2, -2, -2, 0, 1],
        ],
        b := [-1, -3./2],
        c := [-4, -3, -7, 0, 0],
        Jb := [3, 4],
        res := [1/4., 1/2., 0, 0, 0]
    ],
    [
        syst := [
            [1, -5, 1, 0],
            [-3, 1, 0, 1],
        ],
        b := [-10, -12],
        c := [0, -6, 1, 0],
        Jb := [2, 3],
        res := [5, 3, 0, 0]
    ],
    [
        syst := [
            [-3, 1, 2, 0],
            [1, -2, 0, 1],
        ],
        b := [-3, -4],
        c := [-3, -3, 2, 1],
        Jb := [2, 3],
        res := [2, 3, 0, 0]
    ],
]

In [7]:
def Dual_simplex(_syst, _b, _c, _Jb):
    n = len(_syst[0])

    _syst = np.array(copy.deepcopy(_syst), dtype=float)
    _b = np.array(_b[:], dtype=float)
    _Jb = np.array(_Jb[:], dtype=int)
    _c = np.array(_c[:], dtype=float)
    

    while True:
        _x = np.array([0] * n, dtype=float)
        _x[_Jb] = _b

        _Ab = _syst[:, _Jb]
        _cb = _c[_Jb]

        _Ab_inv = np.linalg.inv(_Ab)

        y = np.matmul(_cb, _Ab_inv)

        not_Jb = np.array([i for i in range(n) if i not in _Jb])

        pseudo_plan = np.matmul(_Ab_inv, _b)

        _x[_Jb] = pseudo_plan

        if all(_x >= 0):
            print("План отпимальный")
            print(f'F{tuple([round(xj, 3) for xj in _x])} = {np.array(_c).dot(_x)}')
            return

        minimum_pseudo_x_index = np.argmin(pseudo_plan)

        delta_y = _Ab_inv[minimum_pseudo_x_index]

        _mu = np.array([np.inf] * n)
        _mu[not_Jb] = np.matmul(delta_y, _syst[:, not_Jb])

        if not any(muj < 0 for muj in _mu):
            print("Задача несовместна")
            return

        sigmas = []
        for j in not_Jb:
            if _mu[j] >= 0:
                sigmas.append(np.inf)
            else:
                sigmas.append((_c[j] - _syst[:, j].dot(y)) / _mu[j])

        sigma0 = min(sigmas)
        if (sigma0 == np.inf):
            print('Целевая функция неограничена сверху')
            return

        sigma0_index = not_Jb[np.argmin(sigmas)]
        _Jb[minimum_pseudo_x_index] = sigma0_index

        y += sigma0 * delta_y

In [8]:
for *data, res in test_cases:
    Dual_simplex(*data)
    print(f'Правильный ответ:\nx = {tuple(res)}\n\n')

План отпимальный
F(0.25, 0.5, 0.0, 0.0, 0.0) = -2.5
Правильный ответ:
x = (0.25, 0.5, 0, 0, 0)


План отпимальный
F(5.0, 3.0, 0.0, 0.0) = -18.0
Правильный ответ:
x = (5, 3, 0, 0)


План отпимальный
F(2.0, 3.0, 0.0, 0.0) = -15.0
Правильный ответ:
x = (2, 3, 0, 0)


